In [ ]:
# Efficient : 2019 Google Brain
# 기존 모델보다 파라미터수를 줄이고 더 정확도를 높임
# 기존 CNN
    # 깊이 : ResNet (50 -> 101)
    # 너비 : WideResNet 각 층의 채널수를 늘린다
    # 해상도 : 224 -> 331
# 깊이 너비 해상도 --> 비율로 동시에 확장
#  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit # 시그모이드 함수와 동일
# -10 ~ 10 까지 0.2 간격
x = np.arange(-10,10,0.2)

# ReLU 함수
# x.clip(0) 0보다 작으면 0, 0보다 크면 x 자신으로 만듦
plt.plot(x.clip(0), label='relu')


# Swish 함수
plt.plot(x,expit(x), label='swish')
plt.legend()

In [ ]:
# relu 함수 : 음수를 0으로 죽임
# swish 함수 : 작은 음수를 허용함

In [ ]:
import torch
import torch.nn  as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm import tqdm

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4,0.4,0.4), std=(0.2023, 0.1994, 0.2010)) #
]) # ^

# 이미지 데이터 로드
trainset = torchvision.datasets.CIFAR10(root='./',train=True,download=True,transform=transform)
testset = torchvision.datasets.CIFAR10(root='./',train=False,download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=64)
classes = ('plane', 'car', 'bird', 'cat', 'dear', 'dog', 'frog', 'horse', 'ship', 'truck')
model = models.efficientnet_b0(weights=None, num_classes=10)
print(model.classifier)

In [ ]:
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10) # ^

```
models.efficientnet_b0(weights=None, num_classes=10) 안의 모습 :
Sequential(
   (0): Dropout(p=0.2, inplace=True)
   (1): Linear(in_features=1280, out_features=10, bias=True)
 )

model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
1. model.classifier[1]로 (1): Linear(in_features=1280, out_features=10, bias=True)를 지정한다.

2. model.classifier[1].in_features 를 1280 에서 10으로 변경한다.

```

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# 손실함수 옵티마이져 : CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

# 기존모델의 가중치 + 사용자의 데이터로 학습 --> fine tuning
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(5):
    batch_loss = 0
    for i, data in enumerate(tqdm(trainloader)):
        optimizer.zero_grad()
        inputs, labels = data[0].to(device), data[1].to(device) # ^

        outputs = model(inputs)
        loss = criterion(outputs, labels) # nn.CrossEntropyLoss().(여기에 매게변수)
        loss.backward()
        optimizer.step()

        batch_loss += loss.item()
    print(f'epoch : {epoch+1} loss : {batch_loss/len(trainloader)}')